In [1]:
import torch
import math
import numpy as np
import torchvision
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
import torch.nn.functional as F
import torch.nn as nn
import time

epochNum = 10
#massive array
#outer layer is by loss func, inners are all by learning rate
results = []
#by lossFunc
byLossFunc = [] 
#long list of em
byLR = []
listTimes = []

lossFuncs = [F.relu, F.leaky_relu, F.tanh, F.sigmoid, F.elu, F.softplus]

trainingTimes = []




for b in range (0, len(lossFuncs)):
    byLossFunc.append([]) 
    trainingTimes.append([])

dataset = MNIST(root = 'data/', download = True, transform = ToTensor())

def splitIndices(n, valPct): #split data set into validation and training set, n is number of images, valPct is number you want to be validation set
    nVal = int(valPct * n) #multiplying to find # of images to make validation
    idxs = np.random.permutation(n) #creates a random permutation of n images from 0 to n-1 in the list of images
    return idxs[nVal:], idxs[:nVal] #picks the first nVal indices to be used for validation set and returns trianing images and validation images split up and shuffled

trainIndices, valIndices, = splitIndices(60000, 0.20)

batchSize = 100

trainSampler = SubsetRandomSampler(trainIndices)
trainDL = DataLoader(dataset, batchSize, trainSampler)

valSampler = SubsetRandomSampler(valIndices)
valDL = DataLoader(dataset, batchSize, valSampler)

def getDefaultDevice():
    if torch.backends.mps.is_available():
        return torch.device('cpu')
    else:
        return torch.device('cpu')

device = getDefaultDevice()
#each nn.Linear object we create, it creates 2 matrices of weights and biases to multiply inputs to get outputs we are going to create 2 so we have a hidden layer in between input and output
for a in range (0, len(lossFuncs)):
    byLR.clear()
    listTimes.clear()
        
    class MnistModel(nn.Module):

        
        #create hidden layer and output layer
        def __init__(self, inSize, hiddenSize1, hiddenSize2, hiddenSize3, hiddenSize4, hiddenSize5, outSize):
            super().__init__()
            self.linear1 = nn.Linear(inSize, hiddenSize1).to(device)
            self.linear2 = nn.Linear(hiddenSize1, outSize).to(device)
            # self.linear3 = nn.Linear(hiddenSize2, hiddenSize3).to(torch.device(device))
            # self.linear4 = nn.Linear(hiddenSize3, hiddenSize4).to(torch.device(device))
            # self.linear5 = nn.Linear(hiddenSize4, hiddenSize5).to(torch.device(device))
            # self.linear6 = nn.Linear(hiddenSize5, outSize).to(torch.device(device))
            

        def forward(self, xb): #xb is batch of data
            #flatten tensors
            xb = xb.view(xb.size(0), -1) #xb.size(0) retains batch size input, -1 makes pytorch calculate so the model is general and can be used with images that dont have 784 pixels
            #immediate outputs
            out = self.linear1(xb)
            #apply activation function

            if lossFuncs[a] == F.softmax:
                 out = lossFuncs[a](out, -2) #stock relu just turns negatives into 0
            else:
                out=lossFuncs[a](out)
            #get predictions using output layer
            out = self.linear2(out)
            # out = F.relu(out)
            # out = self.linear3(out)
            # out = F.relu(out)
            # out = self.linear4(out)
            # out = F.relu(out)
            # out = self.linear5(out)
            # out = F.relu(out)
            # out = self.linear6(out)
           
            return out
        
        
    inputSize = 784
    numClasses = 10

    model = MnistModel(784, 256, 64, 32, 16, 12, 10) #dw about it, kthese are not the real values


    for images, labels in trainDL:
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        #print('Loss:', loss.item())
        break



    def toDevice(data, device):
        if isinstance(data, (list, tuple)): #if there are multiple tensors, move all to device
            return [toDevice(x, device) for x in data] 
        return data.to(device, non_blocking = True)

    # for images, labels in trainDL:
    #     print(images.shape)
    #     images = toDevice(images, device)
    #     print(images.device)
    #     break

    #moves data to specific device in batches so not all at once
    class DeviceDataLoader():
        #wwraps a dataloader to move data to device
        def __init__(self, DL, device):
            self.DL = DL
            self.device = device
        #yields batch of data after moving it to device
        def __iter__(self):
            for b in self.DL:
                yield toDevice(b, self.device)  
        
        def __len__(self):
            return len(self.DL)  # number of batches

    # for xb, yb in valDL:
    #     print('xb.device:', xb.device)
    #     print('yb:', yb)
    #     break

    def lossBatch(model, lossFunc, xb, yb, opt = None, metric = None):
        preds = model(xb) #gen predictions
        loss = lossFunc(preds, yb)

        if opt is not None:
            loss.backward()
            opt.step()
            opt.zero_grad()

        metricResult = None
        if metric is not None:
            metricResult = metric(preds, yb)
        
        return loss.item(), len(xb), metricResult


    #calculates overall loss and a metric and also outputs total size of all batches together 
    def evaluate(model, lossFn, validDl, metric = None):
    
        # if len(byLR) == epochNum:
        #     results.append(byLR)
        #     byLR.clear()


        with torch.no_grad(): #dont need to compute gradients with validation set, only for evaluation
            results = [lossBatch(model, lossFn, xb, yb, metric = metric) 
                                        for xb, yb in validDl] #passes each batch through the model
            
            #seperate
            losses, nums, metrics = zip(*results)

            #total size is sum of all batch sizes
            total = np.sum(nums)

            avgLoss = np.sum(np.multiply(losses, nums)) / total #avg loss
            avgMetric = None

            if metric is not None:
                #avg metric of assessment across all batches
                avgMetric = np.sum(np.multiply(metrics, nums)) / total
                    
    
        byLR.append(round(avgMetric, 4))


        return avgLoss, total, avgMetric


    def fit(epochs, lr, model, lossFN, trainDL, validDL, metric = None, optFN = None):
        losses, metrics, = [], []

        if optFN is None: optFN = torch.optim.SGD
        opt = optFN(model.parameters(), lr = lr)

        for epoch in range(epochs):
            time1 = time.perf_counter()
            for xb, yb in trainDL:
                lossBatch(model, lossFN, xb, yb, opt)

            result  = evaluate(model, lossFN, validDL, metric)
            valLoss, total, valMetric = result

            losses.append(valLoss)
            metrics.append(valMetric)

            if metric is None:
                    print('Epoch [{}/{}], Loss: {:.4f}'
                    .format(epoch+1, epochs, valLoss))
            
            else:
                print('Epoch [{}/{}], Loss: {:.4f}, {}: {:.4f}'
                .format(epoch+1, epochs, valLoss, "Accuracy", valMetric))
            time2 = time.perf_counter()

            epochTime = time2-time1
            listTimes.append(round(epochTime, 3))

        return losses, metrics

    def accuracy(outputs, labels):
        _, preds = torch.max(outputs, dim = 1)
        return torch.sum(preds == labels).item() / len(preds)

    model = MnistModel(inputSize, hiddenSize1 = 32, hiddenSize2=0, hiddenSize3=0, hiddenSize4=0, hiddenSize5=0, outSize = numClasses)
    toDevice(model, device)


    # losses, metrics = fit(epochNum, 0.1, model, F.cross_entropy, trainDL, valDL, accuracy)

    learningRates = [0.01,0.05,0.1,0.5,1]




    def createArray():
        for x in range(0, len(learningRates)):
            #run this for every learning rate
            fit(epochNum, learningRates[x], model, F.cross_entropy, trainDL, valDL, accuracy)
        #loop through byLR

        for y in range(0, len(learningRates)):
            tempAcc = []
            tempTimes = []
            for z in range (0, epochNum):
                tempAcc.append(byLR[y+z])
                tempTimes.append(listTimes[y+z])
            
            byLossFunc[a].append(tempAcc)
            trainingTimes[a].append(tempTimes)
        

    createArray()
print(byLossFunc)
print(trainingTimes)


# print("lr="+lr)
# print("epochs="+epochNum)


#originally only had one hidden layer, but changed to two which slightly increased accuracy from 98.4 to 99 :) (2 hidden layers of 64, 32)
#with 3 hidden layers (64, 32, 16) i got to 99.7 in 20 epochs

#(act func is relu)with 4 hidden layers (128, 64, 32, 16) i got to 99.91 in 20 epochs, loss and accuracy jumped around when it flattened out within a percent or so even though learning rate was only 0.08
# with act funcs of sigmoid, relu, leakyrelu and relu, accuracy dropped to 97.81 after 20 epochs

#20 epochs with 32 64 128 32 got 99.91

#20 epochs with 4 hidden layers, 512 1024 256 64 i got accuracy of 100% starting at 16 epochs, ended with a loss of 0.0004, it is important to note
#that it took much longer to train (7 minutes ish)

#same excpet config is 1024 2048 512 64 got accuray of 100% on the 16th epoch as well, ended with loss of 0.0003 took same training time

#20 epochs hdiden layer sizes of 1024 1536 768 256 64 learning rate of 0.1 took a long time to train () because of extra hidden layer and last layer using sigmoid
#anywhere i use sigmoid takes way longer to train after 17 epochs i got 99.94 pretty bad took like 15 - 20 minutes to run

# 20 epoch same sizes but all relu got 100% on epoch 14 and loss of 0.0001 at epoch 18 and ended with 0.0001, however, it took 17 minutes to train



Epoch [1/10], Loss: 0.8867, Accuracy: 0.8148
Epoch [2/10], Loss: 0.5409, Accuracy: 0.8661


KeyboardInterrupt: 